In [1]:
# 必要なモジュールをインポート
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from openai.types.chat import ChatCompletionToolParam
from tavily import TavilyClient

# 環境変数の取得
load_dotenv("../.env")

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# tavily検索用APIキーの取得
TAVILY_API_KEY = os.environ['TAVILY_API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [2]:
# 検索結果を返す関数の作成
def get_search_result(question):
    client = TavilyClient(api_key=TAVILY_API_KEY)
    response = client.search(question)
    return json.dumps({"result": response["results"]})

In [3]:
# テスト用コード
ret = get_search_result("東京駅のイベントを教えて")
json.loads(ret)

{'result': [{'url': 'https://ekitan.com/event/station-2590',
   'title': '東京駅周辺のイベント - 駅探',
   'content': '## 東京駅のイベント一覧. ### 条件指定. * #### 日本の桜名所100選\u3000赤城南面千本桜！菜の花・桜・花桃の中を駆け抜けるトロッコわたらせ渓谷号. + 二重橋前駅／東京駅／大手町駅(東京)駅. 期間2026年2月17日(火)～2月18日(水) このイベントは終了しました. * #### 平日昼間\u3000初心者向け 山手線1周ラン 約10～42キロ（キロ約7～8分）. + 東京駅／二重橋前駅／京橋駅(東京)駅. 期間2026年2月6日(金)～2月14日(土) このイベントは終了しました. + 二重橋前駅／東京駅／大手町駅(東京)駅. * #### 山手線1周ウォーク 約10～42キロ（時速3～4キロ） 11月. + 東京駅／二重橋前駅／京橋駅(東京)駅. * #### 山手線1周ウォーク 約10～42キロ（時速3～4キロ） 5月. + 東京駅／二重橋前駅／京橋駅(東京)駅. * #### 初心者向け 山手線1周ラン 約10～42キロ（キロ約7～8分） 4月. + 東京駅／二重橋前駅／京橋駅(東京)駅. * #### 初心者向け 山手線1周ラン 約10～42キロ（キロ約7～8分） 4月. + 東京駅／二重橋前駅／京橋駅(東京)駅. * #### 初心者向け ゲートブリッジラン 約15～60キロ（キロ約7～8分）. + 東京駅／二重橋前駅／京橋駅(東京)駅. ### 近くの駅のイベント一覧. ### 東京駅沿線のイベント一覧. ### 東京駅の周辺情報. ### 東京駅の路線図・停車駅. ### 東京駅の運行情報. ### 東京駅発の人気の区間(乗換案内). ### 関連サービス. ### **おすすめ記事**.',
   'score': 0.99993396,
   'raw_content': None},
  {'url': 'https://iko-yo.net/events?prefecture_ids%5B%5D=13&tags%5B%5D=%E6%9D%B1%E4%BA%AC%E9%A7%85&term=1',
   '

In [4]:
# ツール定義
def define_tools():
    print("------define_tools(ツール定義)------")
    return [
        ChatCompletionToolParam({
            "type": "function",
            "function": {
                "name": "get_search_result",
                "description": "最近一ヵ月のイベント開催予定などネット検索が必要な場合に、質問文の検索結果を取得する",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {"type": "string", "description": "質問文"},
                    },
                    "required": ["question"],
                },
            },
        })
    ]

In [5]:
# 言語モデルへの質問を行う関数
def ask_question(question, tools):
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": question}],
        tools=tools,
        tool_choice="auto",
    )
    return response

In [6]:
# ツール呼び出しが必要な場合の処理を行う関数
def handle_tool_call(response, question):
    # 関数の実行と結果取得
    tool = response.choices[0].message.tool_calls[0]
    function_name = tool.function.name
    arguments = json.loads(tool.function.arguments)
    function_response = globals()[function_name](**arguments)

    # 関数の実行結果をmessagesに加えて再度言語モデルを呼出
    response_after_tool_call = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "user", "content": question},
            response.choices[0].message,
            {
                "tool_call_id": tool.id,
                "role": "tool",
                "content": function_response,
            },
        ],
    )
    return response_after_tool_call

In [7]:
# ユーザーからの質問を処理する関数
def process_response(question, tools):
    response = ask_question(question, tools)

    if response.choices[0].finish_reason == 'tool_calls':
        # ツール呼出の場合
        final_response = handle_tool_call(response, question)
        return final_response.choices[0].message.content.strip()
    else:
        # 言語モデルが直接回答する場合
        return response.choices[0].message.content.strip()

In [8]:
tools = define_tools()

# 言語モデルが直接回答できる質問
question = "東京都と沖縄県はどちらが広いですか？"
response_message = process_response(question, tools)
print(response_message)

------define_tools(ツール定義)------
東京都と沖縄県の面積を比較すると、沖縄県の方が広いです。

- 東京都の面積は約2,194平方キロメートルです。
- 沖縄県の面積は約2,271平方キロメートルです。

したがって、沖縄県は東京都よりも広い地域を持っています。


In [9]:
tools = define_tools()

# ツール呼出が必要な質問
question = "東京駅のイベントについて、最近1ヶ月以内の検索結果を教えてください"
response_message = process_response(question, tools)
print(response_message)

------define_tools(ツール定義)------
最近1ヶ月以内の東京駅でのイベント情報をいくつかご紹介します。

1. **行幸地下ギャラリー**  
   - 開催場所: 丸の内スぺースアンドメディア  
   - 内容: 様々なアート展示が行われています。詳細については[こちら](https://www.marunouchispacemedia.jp/e/spaces/5)をご覧ください。

2. **バターに関するイベント**  
   - 開催期間: 3月2日まで  
   - 内容: 東京駅でバターに関するイベントが開催中です。様々なバター商品や試食が楽しめます。詳細は[こちら](https://x.com/butterbutlerjp/status/1349624120082763778)を参照。

3. **食事イベント**  
   - 開催場所: 東京駅周辺  
   - 内容: 50代、60代の方向けの特別な食事会が予定されています。詳細情報は[こちら](https://www.ya-7.com/event_info.php?EventNum=24461&ya7_domain_name=ya7)で確認できます。

4. **東京駅改札内のグランスタ**  
   - 開催期間: 約1ヶ月間  
   - 内容: 改札内でのPOP UPイベントが行われており、特別な商品の販売がされています。詳細は[こちら](https://prtimes.jp/main/html/rd/p/000000064.000152494.html)を参照してください。

これらのイベントは、東京駅周辺で様々な体験ができる機会を提供していますので、興味のある方はぜひ参加してみてください。


In [10]:
# チャットボットへの組み込み
tools = define_tools()

messages=[]

while(True):
    # ユーザーからの質問を受付
    question = input("メッセージを入力:")
    # 質問が入力されなければ終了
    if question.strip()=="":
        break
    display(f"質問:{question}")

    # メッセージにユーザーからの質問を追加
    messages.append({"role": "user", "content": question.strip()})
    # やりとりが8を超えたら古いメッセージから削除
    if len(messages) > 8:
        del_message = messages.pop(0)

    # 言語モデルに質問
    response_message = process_response(question, tools)

    # メッセージに言語モデルからの回答を追加
    print(response_message, flush=True)
    messages.append({"role": "assistant", "content": response_message})

print("\n---ご利用ありがとうございました！---")

------define_tools(ツール定義)------


'質問:こんにちわ'

こんにちは！今日はどのようにお手伝いできますか？

---ご利用ありがとうございました！---
